# List Comparison
---
* Compares two arrays for the most likley match
* It then outputs the results into a dataframe

In [42]:
# import packages
import pandas as pd
import difflib as dl

* Read in the data and create two dataframes

In [2]:
# filepath
data_file = '/Users/stephenminter/Desktop/FruitSampleData.xlsx'

#read in filepath and select base tab
df_base = pd.read_excel(data_file, sheet_name= 'base')
df_comparison = pd.read_excel(data_file, sheet_name= 'comparison')

* adds a lower case column for each array, this is to make the search more accruate.

In [3]:
# add a new column for each dataframe, set as a lowercase version of the data
df_base['lower_case'] = df_base.iloc[:, 0].str.lower()
df_comparison['lower_case'] = df_comparison.iloc[:, 0].str.lower()

* create a list from each lower case array, used in the for loop

In [4]:
# turning into a list, this will be used for the comparison data
list_base = df_base[df_base.columns[0]].to_list()
list_base_lower = df_base['lower_case'].to_list()
list_comparison = df_comparison['lower_case'].to_list()

# remove duplicates from list_comparison
list_comparison = list(dict.fromkeys(list_comparison))

* for loop

In [5]:
# create an empty list and for loop
list_accumulation = []

for word in list_base_lower:
    matched_words = dl.get_close_matches(word,list_comparison, 3)
    list_accumulation.append(matched_words)

df = pd.DataFrame(list(zip(list_base, list_accumulation)))
df

,0,1
0,Apple,"[apple, apple man]"
1,Orange,[orange]
2,Pear,[pear]
3,Banana,[banana]
4,StarFruit,[starfruit]
5,Plum,[plum]
6,StrawBerry,"[strawberry, raseberry, berry]"
7,RaseBerry,"[raseberry, strawberry, berry]"
8,AccesPayBkaBla,[accespa]
9,stephen minter,[stephen]


In [6]:
#split_df = pd.DataFrame(df[1].to_list(), columns = ['match_01', 'match_02', 'match_03'])

# split the list into new columns
split_df = pd.DataFrame(df[1].to_list())
# concat both dataframes
df_output = pd.concat([df[0], split_df], axis=1)
# rename dataframes
df_output.columns = ['base', 'match_01', 'match_02', 'match_03']
df_output

,base,match_01,match_02,match_03
0,Apple,apple,apple man,None
1,Orange,orange,None,None
2,Pear,pear,None,None
3,Banana,banana,None,None
4,StarFruit,starfruit,None,None
5,Plum,plum,None,None
6,StrawBerry,strawberry,raseberry,berry
7,RaseBerry,raseberry,strawberry,berry
8,AccesPayBkaBla,accespa,None,None
9,stephen minter,stephen,None,None


* create a function to apply sequence matcher over two columns

In [7]:
def apply_sm(s, c1, c2): 
    return dl.SequenceMatcher(None, s[c1], s[c2]).ratio()

* need to compare against a lowercase base

In [8]:
df_output['base_lower'] = df_output['base'].str.lower()
df_output = df_output[['base', 'base_lower', 'match_01', 'match_02', 'match_03']]
df_output.fillna("", inplace= True)

* add percentage likness for each column

In [9]:
df_output['match_01_perc'] = df_output.apply(apply_sm, c1='base_lower', c2='match_01', axis=1).round(3)
df_output['match_02_perc'] = df_output.apply(apply_sm, c1='base_lower', c2='match_02', axis=1).round(3)
df_output['match_03_perc'] = df_output.apply(apply_sm, c1='base_lower', c2='match_03', axis=1).round(3)
df_output

,base,base_lower,match_01,match_02,match_03,match_01_perc,match_02_perc,match_03_perc
0,Apple,apple,apple,apple man,,1.000,0.714,0.000
1,Orange,orange,orange,,,1.000,0.000,0.000
2,Pear,pear,pear,,,1.000,0.000,0.000
3,Banana,banana,banana,,,1.000,0.000,0.000
4,StarFruit,starfruit,starfruit,,,1.000,0.000,0.000
5,Plum,plum,plum,,,1.000,0.000,0.000
6,StrawBerry,strawberry,strawberry,raseberry,berry,1.000,0.737,0.667
7,RaseBerry,raseberry,raseberry,strawberry,berry,1.000,0.737,0.714
8,AccesPayBkaBla,accespaybkabla,accespa,,,0.667,0.000,0.000
9,stephen minter,stephen minter,stephen,,,0.667,0.000,0.000
